In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import os
from collections import Counter
from collections import defaultdict
from tqdm import tqdm
from operator import itemgetter
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.IndependentCascadesModel as ids
import ndlib.models.epidemics.ThresholdModel as th
import IPython.display as ipd

# 0. Set Parameters

In [2]:
dataset = "CA-HepTh-2"
numOfNodes = 9877
f = 0.8 # percentage to trim
diffP = 0.5
sizeOfSeed = 100
# number of generated graphs
numOfSim1 = 1000
# --------- IM algorithms prameteres ---------
# Static Greedy
R = 200
# IM rank
L = 1
iters = 10
# --------- Diff test parameteres ---------
# Type of IM seed
numOfSims = 3
numOfIters = 100

# 1. ID Checker
***checks the ids of the input graph***<br/>
This script checks that if the IDs of the node start with 1 and end in the number of nodes

In [ ]:
adjList = np.loadtxt("Data/1 Raw Datasets/input.csv", delimiter=',')
np.set_printoptions(threshold=np.nan)
np.set_printoptions(suppress=True)
np.sort(np.unique(adjList))

# 2. ID fixer
***Fix IDs if the previous step indicate they are corrupted***<br/>
The IDs are from 1 to number of nodes after this script

In [5]:
adjList1 = np.loadtxt("Data/1 Raw Datasets/input.csv", delimiter=',')

allNodes = np.sort(np.unique(adjList1))
numOfNodes = np.shape(allNodes)[0]
for i in range(0,numOfNodes):
    np.place(adjList1,adjList1==allNodes[i],i+1)

df = pd.DataFrame(adjList1)
df = df.astype(np.int64)

df.to_csv("Data/2 Fixed Id datasets/input.csv",header=None,index=False)

# 3. Reducer
***Trim a percent of the edges of the graph***<br/>
Delete a number of edges and save all the reduced graphs duplicated and non-duplicated versions<br/>
***--> Be careful with every run of this script a new set of nodes are deleted <--***


In [6]:
original = pd.read_csv("Data/2 Fixed Id datasets/input.csv").values
numOfEdges = np.shape(original)[0]


# make the non-duplicated version of the input original graph
without_repeat = set()
for e in original:
    if (e[0], e[1]) in without_repeat or (e[1], e[0]) in without_repeat:
        pass
    else:
        without_repeat.add((e[0], e[1]))


# convert set to numpy array
# ***it is the non-dulicated version***
original = []
for e in without_repeat:
    original.append(list(e))
original = np.array(original)


# trim the desired percentage of the nodes
size = len(original)
idx = np.random.choice(np.arange(size), int(f*size), replace=False)
deleted = np.array(list(set(np.arange(size)) - set(idx)))

reducedGraph = original[idx]
deletedEdges = original[deleted]


# save the reduced graphs and deleted edges graph non-duplicated version
np.savetxt("Data/3 Reduced/reducedGraph.csv",reducedGraph, fmt='%d',delimiter=',')
np.savetxt("Data/3 Reduced/deletedEdges.csv",deletedEdges, fmt='%d',delimiter=',')
# build the reduced graphs and deleted edges graph duplicated version
reducedGraphDup = np.vstack([reducedGraph[:, [1,0]], reducedGraph])
deletedEdgesDup = np.vstack([deletedEdges[:, [1,0]], deletedEdges])
# save the reduced graphs and deleted edges graph duplicated version
np.savetxt("Data/3 Reduced/reducedGraphDup.csv",reducedGraphDup, fmt='%d',delimiter=',')
np.savetxt("Data/3 Reduced/deletedEdgesDup.csv",deletedEdgesDup, fmt='%d',delimiter=',')


# 4 Read Adjcency list to R
***4 Read the R adj list into an R model***

In [8]:
!Rscript 4\ Read\ R\ Adj\ Input.R {numOfNodes}

Loading required package: tergm
Loading required package: ergm
Loading required package: network
network: Classes for Relational Data
Version 1.13.0.1 created on 2015-08-31.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


ergm: version 3.9.4, created on 2018-08-15
Copyright (c) 2018, Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Carter T. Butts, University of California -- Irvine
                    Steven M. Goodreau, University of Washington
                    Pavel N. Krivitsky, University of Wollongong
           

# 5 ERGM Model Builder
***5 build the ERGM model from the adj list***<br/>
***--> Be careful with every run of this script a new model is built<--***<br/>

In [9]:
!Rscript 5\ Build\ ERGM\ Model.R

[1] "This script is in charge of building the \n  ergm model"
Loading required package: tergm
Loading required package: ergm
Loading required package: network
network: Classes for Relational Data
Version 1.13.0.1 created on 2015-08-31.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


ergm: version 3.9.4, created on 2018-08-15
Copyright (c) 2018, Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Carter T. Butts, University of California -- Irvine
                    Steven M. Goodreau, University of Washington
              

# 6. Generate Random Networks
***6 generate simulated network from the ERGM Model***<br/>
***--> Be careful with every run of this script a new set of graphs are generated<--***

In [10]:
!Rscript 6\ Simulating\ Similar\ Networks.R {numOfSim1}

[1] "This script is in charge of making \n  doing the ergm simulations"
Loading required package: tergm
Loading required package: ergm
Loading required package: network
network: Classes for Relational Data
Version 1.13.0.1 created on 2015-08-31.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


ergm: version 3.9.4, created on 2018-08-15
Copyright (c) 2018, Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Carter T. Butts, University of California -- Irvine
                    Steven M. Goodreau, University of Washington
    

# 7. Link Prediction
***generate link predicted network***<br/>
average over all of them<br/>


In [3]:
# list the files in the simulated graphs directory
files  = os.listdir("Data/6 Simulated Networks/")
adjLists = [] #containing whole 1000 generated graphs

# read the grpahs adjacency lists
for file in files:
    adjLists = adjLists + [np.genfromtxt("Data/6 Simulated Networks/"+file, delimiter=",",dtype=np.int64)]




# ### count the number of each edge repeat in the whole simulated nets set
c = defaultdict(int)
for adj in tqdm(adjLists):
    ad1 = map(lambda x: (x[0], x[1]), adj)
    for i in ad1:
        c[i] += 1
    ad1 = map(lambda x: (x[1], x[0]), adj)
    for i in ad1:
        c[i] += 1


# ### read the reduced and deleted edges from their file
# (***this files are with duplicate edges***)
reduced = np.genfromtxt("Data/3 Reduced/reducedGraphDup.csv", delimiter=",",dtype=np.int64)
deleted1 = np.genfromtxt("Data/3 Reduced/deletedEdgesDup.csv", delimiter=",",dtype=np.int64)

# ### sorted array to find the top most repeated edges in the simulated networks
# ***we aim to add most repeated with the same number of deleted edges***
# ***in most repeated there are duplicate edges***
most_repeated = sorted(c, key=lambda x: c[x], reverse=True)


# convert numpy array to set
reduced_set = set(map(lambda x: (x[0], x[1]), reduced))

to_add = set()
size1 = deleted1.shape[0]//2
for e in most_repeated:
    if not e in reduced_set:
        if e in to_add or (e[1], e[0]) in to_add or e[0] > numOfNodes or e[1] > numOfNodes:
            continue
        to_add.add(e)
        size1 -= 1
        if size1 == 0:
            break
print("The number of added edge to the reduced graph: "+ str(len(to_add)) )


# ### build the nonDuplicated verstion of reduced
reducedNonDup = set()
for e in reduced_set:
    if (e in reducedNonDup) or ((e[1], e[0]) in reducedNonDup):
        pass
    else:
        reducedNonDup |= {e}


# ### adding the added edges to reduced graph
reducedNonDup.update(to_add)

actual_added = to_add - set(map(lambda x: (x[0], x[1]), deleted1))

print("The difference between actual deleted edges and the added ones is: " + str(len(actual_added)))

# ### to remeber
# ***reducedNonDup: is the graph with added edges and it is obviously non-dup***
# ***to_add: is non-duplicated***
# ***actual_added: is non-duplicated***

def save_edge_set(fname, edge_set):
    with open(fname, 'w') as of:
        for e in edge_set:
            of.write("%d,%d\n"%e)

save_edge_set("Data/7 Link Predicted Network/augmented.csv",reducedNonDup)


100%|██████████| 1000/1000 [00:36<00:00, 27.53it/s]


The number of added edge to the reduced graph: 5200
The difference between actual deleted edges and the added ones is: 3748


# 8. Non-duplicated Builder
***generate non-duplicated version of the original graph***

In [4]:

def save_edge_set(fname, edge_set):
    with open(fname, 'w') as of:
        for e in edge_set:
            of.write("%d,%d\n"%e)


duplicated = np.genfromtxt("Data/2 Fixed Id datasets/input.csv", delimiter=",",dtype=np.int64)
duplicated_set = set(map(lambda x: (x[0], x[1]), duplicated))

nonDup = set()
for e in tqdm(duplicated_set):
    if (e in nonDup) or ((e[1], e[0]) in nonDup):
        pass
    else:
        nonDup |= {e}

        
save_edge_set("Data/8 Original Without Duplicates/original.csv",nonDup)


100%|██████████| 51971/51971 [00:00<00:00, 997936.09it/s]


# 9. Random Graph Builder
***build a random graph with the same size as the augmented and the original graph***<br/>
***--> Be careful with every run of this script a new set of graphs are generated<--***

In [5]:
def randomEdge(numOfNodes):
    return (np.random.randint(1,numOfNodes+1),random.randint(1,numOfNodes+1))

def save_edge_set(fname, edge_set):
    with open(fname, 'w') as of:
        for e in edge_set:
            of.write("%d,%d\n"%e)


reduced1 = np.genfromtxt("Data/3 Reduced/reducedGraph.csv",delimiter=',',dtype=int)
nOfNs2Gen = len(np.genfromtxt("Data/3 Reduced/deletedEdges.csv",delimiter=',',dtype=int))


reduced_set1 = set(map(lambda x : (x[0],x[1]),reduced1))

size2 = nOfNs2Gen
while size2:
    e = randomEdge(numOfNodes)
    if (not e in reduced_set1) and (not (e[1],e[0]) in reduced_set1):
        reduced_set1.add(e)
        size2 -=1


save_edge_set("Data/9 Random Graph/random.csv",reduced_set1)


# 10. Diffusion Added
***Add diffusion probability to the graphs*** <br/>
The three augmented, random and original without duplicates are built at this step

In [6]:
augmented = np.genfromtxt("Data/7 Link Predicted Network/augmented.csv",delimiter=',',dtype = int)
original1 = np.genfromtxt("Data/8 Original Without Duplicates/original.csv",delimiter=',',dtype = int)
random1 = np.genfromtxt("Data/9 Random Graph/random.csv",delimiter=',',dtype = int)


def addProbs(adj,prefix):
    with open("Data/10 IM ready Graphs with Diffusion Probabilities/Cenrality/" + prefix + ".csv",'w') as of:
        for x,y in adj:
            of.write("%d\t%d\t%.1f\n"%(x,y,diffP))
    with open("Data/10 IM ready Graphs with Diffusion Probabilities/IM/" + prefix + ".csv",'w') as of:
        of.write("%d\t%d\n"%(numOfNodes,len(adj)))
        for x,y in adj:
            of.write("%d\t%d\t%.1f\n"%(x,y,diffP))
            

addProbs(augmented,"augmented")
addProbs(original1,"original")
addProbs(random1,"random")


# 11.1 IM (Centralities)
***compute centralities of the graphs***<br/>
Betweenness and PageRank centralities are used.

In [11]:
def intt(x):
    return int(float(x))

def write2file(file,seed):
    f= open(file,"w+")
    for item in seed:
         f.write(str(item) + "\n")
    f.close()

augmented1 = nx.read_weighted_edgelist("Data/10 IM ready Graphs with Diffusion Probabilities/Cenrality/augmented.csv",nodetype=intt)
original2 = nx.read_weighted_edgelist("Data/10 IM ready Graphs with Diffusion Probabilities/Cenrality/original.csv",nodetype=intt)
random2 = nx.read_weighted_edgelist("Data/10 IM ready Graphs with Diffusion Probabilities/Cenrality/random.csv",nodetype=intt)

def computeCentrality(g , graphType):
#     betweenness centrality
    BetCen = nx.betweenness_centrality(g)
    BetCenSeed = sorted(BetCen.items(), key=itemgetter(1), reverse=True)[:sizeOfSeed]
    BetCenSeed = list(map(lambda x: x[0], BetCenSeed))
    write2file("Data/11 IM stage results/Betweenness Centrality/" + graphType + ".csv",BetCenSeed)
#     PageRank Centralilty
    PRCen = nx.pagerank(g)
    PRCenSeed = sorted(PRCen.items(), key=itemgetter(1), reverse=True)[:sizeOfSeed]
    PRCenSeed = list(map(lambda x: x[0], PRCenSeed))
    write2file("Data/11 IM stage results/PageRank Centrality/" + graphType + ".csv",PRCenSeed)

    
computeCentrality(augmented1,"augmented")
computeCentrality(original2,"original")
computeCentrality(random2,"random")

# 11.2. random seed

In [32]:
# def computeRandomSeed(graphType):
randSeed = np.random.randint(1,numOfNodes,(sizeOfSeed,1))
np.savetxt("Data/11 IM stage results/Random/original.csv",randSeed,fmt="%d")
np.savetxt("Data/11 IM stage results/Random/augmented.csv",randSeed,fmt="%d")
np.savetxt("Data/11 IM stage results/Random/random.csv",randSeed,fmt="%d")

# 11.2. IM (IMs)
***compute results of the IM methods***<br/>
IMrank and Static Greedy methods have been used.

In [15]:
!./Static\ Greedy Data/10\ IM\ ready\ Graphs\ with\ Diffusion\ Probabilities/IM/augmented.csv {sizeOfSeed} {R} bsg
!tail -n+5 BasicStaticGreedy_R{R}_k{sizeOfSeed}.txt > Data/11\ IM\ stage\ results/Static\ Greedy/augmented.csv
!rm BasicStaticGreedy_R{R}_k{sizeOfSeed}.txt

network info:9877  25998


In [16]:
!./Static\ Greedy Data/10\ IM\ ready\ Graphs\ with\ Diffusion\ Probabilities/IM/original.csv {sizeOfSeed} {R} bsg
!tail -n+5 BasicStaticGreedy_R{R}_k{sizeOfSeed}.txt > Data/11\ IM\ stage\ results/Static\ Greedy/original.csv
!rm BasicStaticGreedy_R{R}_k{sizeOfSeed}.txt

network info:9877  25998


In [17]:
!./Static\ Greedy Data/10\ IM\ ready\ Graphs\ with\ Diffusion\ Probabilities/IM/random.csv {sizeOfSeed} {R} bsg
!tail -n+5 BasicStaticGreedy_R{R}_k{sizeOfSeed}.txt > Data/11\ IM\ stage\ results/Static\ Greedy/random.csv
!rm BasicStaticGreedy_R{R}_k{sizeOfSeed}.txt

network info:9877  25998


In [22]:
!./IMrank Data/10\ IM\ ready\ Graphs\ with\ Diffusion\ Probabilities/IM/original.csv {sizeOfSeed} {L} {iters} PageRank
!tail -n+5 IMRank_k{sizeOfSeed}_l{L}_LOOP{iters}_irPageRank.txt > Data/11\ IM\ stage\ results/IMrank/original.csv
!rm IMRank_k{sizeOfSeed}_l{L}_LOOP{iters}_irPageRank.txt

network info:9877  25998


In [23]:
!./IMrank Data/10\ IM\ ready\ Graphs\ with\ Diffusion\ Probabilities/IM/augmented.csv {sizeOfSeed} {L} {iters} PageRank
!tail -n+5 IMRank_k{sizeOfSeed}_l{L}_LOOP{iters}_irPageRank.txt > Data/11\ IM\ stage\ results/IMrank/augmented.csv
!rm IMRank_k{sizeOfSeed}_l{L}_LOOP{iters}_irPageRank.txt

network info:9877  25998


In [24]:
!./IMrank Data/10\ IM\ ready\ Graphs\ with\ Diffusion\ Probabilities/IM/random.csv {sizeOfSeed} {L} {iters} PageRank
!tail -n+5 IMRank_k{sizeOfSeed}_l{L}_LOOP{iters}_irPageRank.txt > Data/11\ IM\ stage\ results/IMrank/random.csv
!rm IMRank_k{sizeOfSeed}_l{L}_LOOP{iters}_irPageRank.txt

network info:9877  25998


# 12. Diffusion Test

In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:





def intt(x):
    return int(float(x))

def buildGraph(adjList,numOfNodes):
    numOfEle = np.shape(adjList)[0]
    g = nx.empty_graph(numOfNodes+1)
    for i in range (0,numOfEle):
        g.add_edge(adjList[i,0],adjList[i,1])
    return (g)



# read the graphs from file
augmentedAdj = np.genfromtxt("Data/10 IM ready Graphs with Diffusion Probabilities/Cenrality/augmented.csv")
originalAdj = np.genfromtxt("Data/10 IM ready Graphs with Diffusion Probabilities/Cenrality/original.csv")
randomAdj = np.genfromtxt("Data/10 IM ready Graphs with Diffusion Probabilities/Cenrality/random.csv")

# build the graphs
augmentedGraph = buildGraph(augmentedAdj,numOfNodes)
originalGraph = buildGraph(originalAdj,numOfNodes)
randomGraph = buildGraph(randomAdj,numOfNodes)

# simulation paprameters
# g: input graph
# seed: input seed
def simulation(g,seed):
    # trend array
    trendArr = []
    # Model selection
    model = ids.IndependentCascadesModel(g)
    # Model Configuration
    config = mc.Configuration()
    # config.add_model_parameter
    config.add_model_initial_configuration("Infected", seed)
    # Setting the edge parameters
    for e in g.edges():
        config.add_edge_configuration("threshold",e , diffP)
    model.set_initial_status(config)

    # Simulation execution
#     iterations = model.iteration_bunch(numOfIters)
        # Simulation execution
    for j in range(0,numOfIters):
        iterations = model.iteration_bunch(1)
        trends = model.build_trends(iterations)
        di = model.status
        infectedTrend = np.array(list(filter(lambda x: di[x] in [1, 2], di)))
        trendArr = trendArr + [infectedTrend]
    di = model.status
    infected = np.array(list(filter(lambda x: di[x] in [1, 2], di)))
    return ((infected,trendArr))
#         print('------------------------------------')
#         print(sum(value == 0 for value in model.status.values()))
#         print(sum(value == 1 for value in model.status.values()))
#         print(sum(value == 2 for value in model.status.values()))
        
# run simulation per graphs

class Averager:
    def __init__(self, ar_size):
        self.sum = np.zeros(ar_size)
        self.count = 0
        
    def update(self, arr):
        self.sum += arr
        self.count += 1
        
    def get(self):
        return self.sum / self.count
    
def tests(IMmethod):
    # set the arrays that keeps the num of infected
    augsAr = np.zeros(numOfSims)
    randAr = np.zeros(numOfSims)
    # read the seed set from the file
    augmentedSeed = np.genfromtxt("Data/11 IM stage results/" + IMmethod + "/augmented.csv",dtype=int)
    originalSeed = np.genfromtxt("Data/11 IM stage results/" + IMmethod + "/original.csv",dtype=int)
    randomSeed = np.genfromtxt("Data/11 IM stage results/" + IMmethod + "/random.csv",dtype=int)
    augTrendsMean = Averager(numOfIters)
    randTrendsMean = Averager(numOfIters)
    for i in tqdm(range(0,numOfSims)):
        # run simulation function
        augmentedInfected,augTrends = simulation(augmentedGraph,augmentedSeed)
        originalInfected,orgTrends = simulation(originalGraph,originalSeed)
        randomInfected,randTrends = simulation(randomGraph,randomSeed)
        augTrendsMean.update(np.array([len(set(x) and set(y)) for x, y in zip(augTrends, orgTrends)]))
        randTrendsMean.update(np.array([len(set(x) and set(y)) for x, y in zip(orgTrends, randTrends)]))
        # compute infected array
        augsAr[i] = len(set(originalInfected) and set(augmentedInfected))
        randAr[i] = len(set(originalInfected) and set(randomInfected))
    return((augsAr,randAr, augTrendsMean.get(), randTrendsMean.get()))

resSaver = open("Data/12 Diffusion Test Results/results.csv", "w")
def saveDifRes(IMmethod,res):
    # save infected during iteration
    np.save("Data/12 Diffusion Test Results/" + IMmethod + "/augmented",res[0])
    np.save("Data/12 Diffusion Test Results/" + IMmethod + "/random",res[1])
    np.save("Data/12 Diffusion Test Results/" + IMmethod + "/augmentedTrend",res[2])
    np.save("Data/12 Diffusion Test Results/" + IMmethod + "/randomTrend",res[3])
    # save infection trend
    # ------------
    resSaver.write(str(np.mean(res[0])) + "," + str(np.mean(res[1])) + "\n")
# get Betweeness Cntrality results
res1 = tests("Betweenness Centrality")
saveDifRes("Betweenness Centrality",res1)
# get PageRank Cntrality results
res2 = tests("PageRank Centrality")
saveDifRes("PageRank Centrality",res2)
# get IMrank Cntrality results
res3 = tests("IMrank")
saveDifRes("IMrank",res3)
# get Static Greedy Cntrality results
res4 = tests("Static Greedy")
saveDifRes("Static Greedy",res4)
# get random results
res5 = tests("Random")
saveDifRes("Random",res5)

resSaver.close()

# print(orgConvfTest)
# print(randConvfTest)



100%|██████████| 3/3 [00:08<00:00,  2.87s/it]


In [2]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [4]:
a = np.load("Data/12 Diffusion Test Results/Betweenness Centrality/augmentedTrend.npy")
b = np.load("Data/12 Diffusion Test Results/Betweenness Centrality/randomTrend.npy")


In [5]:
a,b

(array([ 100.        , 1292.33333333, 3442.33333333, 5163.66666667,
        6048.33333333, 6392.        , 6525.        , 6583.33333333,
        6605.        , 6614.        , 6616.66666667, 6619.        ,
        6620.33333333, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666667, 6620.66666667,
        6620.66666667, 6620.66666667, 6620.66666